# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

## Movie distribution

First solve Problem 3 from HW3-theoretical. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [19]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [20]:
for state in G.nodes():
    if state != 'CA':
        G.node[state]['demand'] = 1
G.node['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [21]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edge[s1][s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [22]:
def flow_with_demands(graph):
    newgraph = graph.copy()
    newgraph.add_node('source')
    newgraph.add_node('terminal')
    
    for node in graph.nodes():
        demand = graph.node[node]['demand']
        if demand < 0:
            newgraph.add_edge('source',node)
            newgraph['source'][node]['capacity'] = -demand
        if demand > 0:
            newgraph.add_edge(node,'terminal')
            newgraph[node]['terminal']['capacity'] = demand
    
    if nx.maximum_flow(newgraph, 'source', 'terminal')[0] < sum(graph.node[node]['demand'] for node in graph.nodes() if graph.node[node]['demand']>0):
            raise nx.NetworkXUnfeasible('NetworkXUnfeasible: No flow satisfying the demands!')
            
    flow_value, flow = nx.maximum_flow(newgraph,'source','terminal')
    flow.pop('source')
    flow.pop('terminal')
    for i in flow:
        try:
            flow[i].pop('source')
        except:
            pass
        try:
            flow[i].pop('terminal')
        except:
            continue
    
    newgraph.remove_node('source')
    newgraph.remove_node('terminal')


    return flow

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [23]:
def divergence(flow):
    network = nx.DiGraph()
    edges = [(s1,s2) for s1 in flow.keys() for s2 in flow[s1].keys()]
    for (s1,s2) in edges:
        network.add_edge(s1,s2)
        network.edge[s1][s2]['flow'] = flow[s1][s2]
    
    for n in network.nodes():
        outflow = 0
        inflow = 0
        for s in network.successors(n):
            outflow = outflow + network.edge[n][s]['flow']
        for p in network.predecessors(n):
            inflow = inflow + network.edge[p][n]['flow']
        network.node[n]['demand'] = inflow - outflow
    
    div = {n : network.node[n]['demand'] for n in network.nodes()}

    return div
    

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [24]:
flow = flow_with_demands(G)
div = divergence(flow)
print "Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes())

Flow satisfies all demands: True
